# Tool Call LLMs
Some of the more exciting use cases for RC is when you want to cede control of the decision making process to the LLM. RC has a suite of tools that you can use to make this process much simpler. Check out an example below. 

In [1]:
from typing import Dict, Any, Set, Type
from pydantic import BaseModel, Field
import src.requestcompletion as rc


In [2]:
class HarshCritic(rc.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a harsh critic of the world and you should analyze the given statement and provide a harsh critique of it. Be very concise and to the point."

    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )

    def create_model(self) -> rc.llm.ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Harsh Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="Harsh_Critic",
            detail="A tool used to critique a statement harshly.",
            parameters={rc.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> rc.library.TerminalLLM:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
        
    

In [3]:
class PositiveCritic(rc.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a positive critic of the world and you should analyze the given statement and provide a positive critique of it. Be very concise and to the point."

    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )

    def create_model(self) -> rc.llm.ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Positive Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="Positive_Critic",
            detail="A tool used to critique a statement positively.",
            parameters={rc.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze")}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> rc.library.TerminalLLM:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)

In [37]:
class DeeperMeaningCritic(rc.library.TerminalLLM):
    @classmethod
    def system_message(cls):
        return "You are a critic of the world and you should analyze the given statement and provide a deep meaning critic of it. Be very concise and to the point."
    
    @classmethod
    def create_model(cls) -> rc.llm.ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")

    def __init__(
            self,
            message_history: rc.llm.MessageHistory
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Deeper Meaning Critic"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="Deeper_Meaning_Critic",
            detail="A tool used to critique a statement.",
            parameters={rc.llm.Parameter("analysis_detail", "string", "The thing you would like to analyze",)}
            
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> rc.library.TerminalLLM:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist)
    

In [39]:
system_message_critic = rc.llm.SystemMessage("You are a critic of the world and you provide comprehensive critiques of the world around you. You should utilize the provided tools to collect specific critiques and structure them before completing your answer.")
Critic = rc.library.tool_call_llm(connected_nodes={HarshCritic, PositiveCritic, DeeperMeaningCritic},
                                  pretty_name="Critic", 
                                  system_message=system_message_critic, 
                                  model=rc.llm.OpenAILLM("gpt-4o"),
                                #   output_type="MessageHistory",
                                  )

In [40]:
template = ("I am writing a short story and I would like to analyze my introduction.\n"
            "\n"
            "Once upon a time there was a little boy who lived in a small village. He was a very kind and generous, but lacked an understanding"
            " of the world around him. He was always looking for ways to help others and make the world a better place. One day, he stumbled upon a"
            " magical book that would change his life forever. The book was filled with stories of adventure and mystery, and the promise of a better"
            " tomorrow.")
with rc.Runner(executor_config=rc.run.ExecutorConfig(timeout=50)) as runner:
    response = await runner.run(Critic, message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)]))


In [41]:
response.answer

assistant: Your introduction beautifully captures the essence of innocence and innate goodness in a child's heart. The little boy's kindness and generosity set a positive example for readers. His discovery of the magical book symbolizes the transformative power of knowledge and imagination, suggesting that even simple beginnings can lead to profound personal growth and a brighter future. The narrative encourages curiosity and the pursuit of understanding, highlighting the potential for positive change when one is open to learning and adventure.

Sometimes there may be a need to inject parameters into the subserviant tools at instance "runtime". This can be done by modifying the node creation method in the parent top level node


In [ ]:
class StructuredSummarizerOutput(BaseModel):
    positive_critique: str = Field(description="The positive critique")
    harsh_critique: str = Field(description="The harsh critique")
    overall_score: float = Field(description="The overall score of the critique (on a scale of 0 to 100)")
    summary: str = Field(description="The summary of all the critiques")
            
class StructureSummarizerCritic(rc.library.StructuredLLM):
    @classmethod
    def system_message(cls):
        return "You are a reviwer tool that summarizes the critiques and structures it."

    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
    ):
        message_history.insert(0, rc.llm.SystemMessage(self.system_message()))
        super().__init__(
            message_history=message_history,
            model=self.create_model(),
        )

    def create_model(self) -> rc.llm.ModelBase:
        return rc.llm.OpenAILLM("gpt-4o")
    
    def output_model(self) -> StructuredSummarizerOutput:
        return StructuredSummarizerOutput
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Structured Summarizer"
    
    # for types which you want to connect to an LLM using our tooling you should implement this method
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="Structure_and_Summarize",
            detail="A tool used to generate summaries of critiques and structure them.",
            parameters={rc.llm.Parameter("positive_critique", "string", "The positive critique"),
                        rc.llm.Parameter("harsh_critique", "string", "The harsh critique"),
                        rc.llm.Parameter("overall_score", "float", "The overall score of the critique. Give only a number between 0 and 100"),
                        rc.llm.Parameter("summary", "string", "The summary of all the critiques")
                        }   
        )
    
    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]) -> rc.library.TerminalLLM:
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(f"Positive Critique: {tool_parameters['positive_critique']}"), 
                                              rc.llm.UserMessage(f"Harsh Critique: {tool_parameters['harsh_critique']}"),
                                              rc.llm.UserMessage(f"Overall Score: {tool_parameters['overall_score']}"),
                                              rc.llm.UserMessage(f"Summary: {tool_parameters['summary']}"),
                                              ])
        return cls(message_hist)
    
    

In [9]:
class GradeLevelHarshCritic(HarshCritic):

    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
            grade_level: int
    ):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    @classmethod
    def prepare_tool(cls, tool_parameters):
        message_hist = rc.llm.MessageHistory([rc.llm.UserMessage(tool_parameters["analysis_detail"])])
        return cls(message_hist, tool_parameters["grade_level"])
        

    def system_message(self):
        return super().system_message() + " You should provide a critique at a grade level of " + str(self.grade_level)


In [10]:
class InjectGradeLevel(Critic):
    def __init__(
            self,
            message_history: rc.llm.MessageHistory,
            grade_level: int
    ):
        super().__init__(message_history=message_history)
        self.grade_level = grade_level

    def create_node(self, tool_name: str, arguments: Dict[str, Any]) -> rc.Node:
        if tool_name == "Harsh_Critic":
            arguments["grade_level"] = self.grade_level
            return GradeLevelHarshCritic.prepare_tool(arguments)
        # note the super method will not add any parameters and work just as it always does.
        return super().create_node(tool_name, arguments)


    def connected_nodes(self) -> Set[Type[rc.Node]]:
        # note that you can add the new node to the connected nodes set
        return {GradeLevelHarshCritic, PositiveCritic, DeeperMeaningCritic}





In [11]:
template = ("I am writing a short story and I would like to analyze my introduction.\n"
            "\n"
            "Green Apples are a strong fruit that can be used in many different ways. They are a great source of vitamins and minerals, and can be eaten or even thrown at others you don't like")
with rc.Runner(executor_config=rc.run.ExecutorConfig(timeout=50)) as runner:
    response = runner.run(InjectGradeLevel(
        message_history=rc.llm.MessageHistory([rc.llm.UserMessage(template)]),
        grade_level=12),
    )


print(response.answer)

AttributeError: 'coroutine' object has no attribute 'answer'